In [46]:
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.3.0


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 데이터 로드

In [2]:
class config():
    def __init__(self):
        self.train_edge_file_path = f'/home/work/kdg_folder/work_kisti/학습데이터/edge_train_vs_1.pkl'
        self.train_node_file_path = f'/home/work/kdg_folder/work_kisti/학습데이터/node_train_vs_1.pkl'
        self.test_edge_file_path = f'/home/work/kdg_folder/work_kisti/학습데이터/edge_test_vs_1.pkl'
        self.test_node_file_path = f'/home/work/kdg_folder/work_kisti/학습데이터/node_test_vs_1.pkl'  
        self.label_col=19      
        self.batch_size=64
        self.in_dim=19
        self.h_dim=256
        self.num_layers=5
        self.out_dim=1

cfg = config()

In [3]:
#train
import pandas as pd

# 딕셔너리를 사용하여 edge 데이터를 저장
edges = {}
nodes = {}
labels = {}

for i in range(1):
    edge_file_path = cfg.train_edge_file_path
    node_file_path = cfg.train_node_file_path
    
    edges[f'edge{i}'] = pd.read_pickle(edge_file_path)
    nodes[f'node{i}'] = pd.read_pickle(node_file_path)
    
    labels[f'label{i}'] = pd.DataFrame(nodes[f'node{i}'].iloc[:,cfg.label_col])
    nodes[f'node{i}'] = nodes[f'node{i}'].iloc[:,:cfg.label_col]

In [5]:
#test
import pandas as pd

# 딕셔너리를 사용하여 edge 데이터를 저장
testedges = {}
testnodes = {}
testlabels = {}

for i in range(1):
    edge_file_path = cfg.test_edge_file_path
    node_file_path = cfg.test_node_file_path
    
    testedges[f'edge{i}'] = pd.read_pickle(edge_file_path)
    testnodes[f'node{i}'] = pd.read_pickle(node_file_path)
    
    testlabels[f'label{i}'] = pd.DataFrame(testnodes[f'node{i}'].iloc[:,cfg.label_col])
    testnodes[f'node{i}'] = testnodes[f'node{i}'].iloc[:,:cfg.label_col]

In [6]:
labels['label0']

,19
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
145780,0.0
145781,0.0
145782,0.0
145783,1.0


In [6]:
for i in range(1):
    edges[f'edge{i}'] = edges[f'edge{i}'].values
    edges[f'edge{i}'] = torch.from_numpy(edges[f'edge{i}']).long()

    nodes[f'node{i}'] = nodes[f'node{i}'].values
    nodes[f'node{i}'] = torch.from_numpy(nodes[f'node{i}']).float()
    
    labels[f'label{i}'] = labels[f'label{i}'].values
    labels[f'label{i}'] = torch.from_numpy(labels[f'label{i}']).float()    



for i in range(1):
    testedges[f'edge{i}'] = testedges[f'edge{i}'].values
    testedges[f'edge{i}'] = torch.from_numpy(testedges[f'edge{i}']).long()

    testnodes[f'node{i}'] = testnodes[f'node{i}'].values
    testnodes[f'node{i}'] = torch.from_numpy(testnodes[f'node{i}']).float()
    
    testlabels[f'label{i}'] = testlabels[f'label{i}'].values
    testlabels[f'label{i}'] = torch.from_numpy(testlabels[f'label{i}']).float()     

In [8]:
labels['label0'].size()

torch.Size([145785, 1])

In [7]:
for i in range(1):
    print(edges[f'edge{i}'].dtype)
    print(nodes[f'node{i}'].dtype)
    print(labels[f'label{i}'].dtype)
    

torch.int64
torch.float32
torch.float32


In [10]:
# print(edge_tensor.size())
# print(node_tensor.size())
# print(label_tensor.size())
# print(edge_tensor.dtype)
# print(node_tensor.dtype)
# print(label_tensor.dtype)

# train data 리스트에 담기

In [21]:
torch.randint(1000,50000,(edges['edge0'].size(1),1))

tensor([[27044],
        [23143],
        [47466],
        ...,
        [35066],
        [37472],
        [43092]])

In [24]:
import torch_geometric
from torch_geometric.data import Data

lst=[]


for i in range(1):
    lst.append(Data(x=nodes[f'node{i}'], edge_index=edges[f'edge{i}'], edge_attr=torch.randint(1000,50000,(edges['edge0'].size(1),1)), y=labels[f'label{i}']))

testlst=[]
for i in range(1):
    testlst.append(Data(x=testnodes[f'node{i}'], edge_index=testedges[f'edge{i}'], edge_attr=torch.randint(1000,50000,(testedges['edge0'].size(1),1)),y=testlabels[f'label{i}']))    


In [25]:
lst[0].edge_attr

tensor([[ 9567],
        [33990],
        [31333],
        ...,
        [22074],
        [47028],
        [ 5095]])

In [26]:
testlst[0].edge_attr

tensor([[11016],
        [47561],
        [49894],
        ...,
        [25775],
        [48480],
        [ 9570]])

In [27]:
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader, NeighborLoader

# train_data = Batch.from_data_list(lst)
# test_loader = NeighborLoader(testlst, batch_size=512, shuffle=True, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)

# train_loader = NeighborLoader(train_data, batch_size=512, shuffle=True, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)
# val_loader = DataLoader(val_dataset, batch_size=2)

# train_loader = DataLoader(Batch.from_data_list(lst), batch_size=64, shuffle=True)
# test_loader = DataLoader(Batch.from_data_list(testlst), batch_size=64, shuffle=False)


# from torch_geometric.data import Batch
# from torch_geometric.loader import DataLoader, NeighborLoader
train_loader = DataLoader(lst, batch_size=16,shuffle=True)
test_loader = DataLoader(testlst, batch_size=16, shuffle=False)


# train_loader = NeighborLoader(Batch.from_data_list(lst), batch_size=cfg.batch_size, shuffle=True, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)
# test_loader = NeighborLoader(Batch.from_data_list(testlst), batch_size=cfg.batch_size, shuffle=False, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)

## PPI

In [28]:
# from torch_geometric.datasets import PPI
# train_dataset = PPI(root=".", split='train')
# val_dataset = PPI(root=".", split='val')
# test_dataset = PPI(root=".", split='test')
 
# from torch_geometric.data import Batch
# from torch_geometric.loader import DataLoader, NeighborLoader
 

# import torch_geometric
# from torch_geometric.data import Data
# lst=[]
# for i in range(20):
#     lst.append(Data(x=train_dataset[i].x, edge_index=train_dataset[i].edge_index, y=train_dataset[i].y[:,0].unsqueeze(1)))

# test_dataset[1].y[:,0].unsqueeze(1)

In [29]:
# testdata1 = Data(x=test_dataset[0].x, edge_index=test_dataset[0].edge_index, y=test_dataset[0].y[:,0].unsqueeze(1))
# testdata2 = Data(x=test_dataset[1].x, edge_index=test_dataset[1].edge_index, y=test_dataset[1].y[:,0].unsqueeze(1))


In [30]:
# train_loader = DataLoader(lst, batch_size=2)
# test_loader = DataLoader([testdata1,testdata2], batch_size=2)

In [31]:
# train_data = Batch.from_data_list(lst)
# train_loader = NeighborLoader(train_data, batch_size=2048, shuffle=True, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)

# test_data = Batch.from_data_list([testdata1,testdata2])
# test_loader = NeighborLoader(test_data, batch_size=2048, shuffle=True, num_neighbors=[20, 10], num_workers=2, persistent_workers=True)

# 모델 만들기 연습

In [32]:
# import torch
# torch.manual_seed(-1)
# import torch.nn.functional as F
# from torch_geometric.nn import SAGEConv

# from sklearn.metrics import f1_score

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# class GraphSAGE(torch.nn.Module):
    
#     def __init__(self, dim_in, dim_h, dim_out):
#         super().__init__()
#         self.sage1 = SAGEConv(dim_in, dim_h)
#         # self.sage2 = SAGEConv(dim_h, dim_h)
#         self.sage3 = SAGEConv(dim_h, dim_out)
        
#     def forward(self, x, edge_index):
#         h = self.sage1(x, edge_index)
#         h = F.elu(h)
#         h = F.dropout(h, p=0.6, training=True)
#         # h = self.sage2(h,edge_index)
#         # h = F.elu(h)
#         h = self.sage3(h, edge_index)
#         # h= F.sigmoid(h)

#         return h

In [33]:
# import torch.nn.functional as F
# from torch.nn import Linear, Dropout
# from torch_geometric.nn import GCNConv, GATv2Conv
# import torch
# torch.manual_seed(-1)

# from torch_geometric.nn import SAGEConv

# from sklearn.metrics import f1_score

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# class GAT(torch.nn.Module):
#   """Graph Attention Network"""
#   def __init__(self, dim_in, dim_h, dim_out, heads=8):
#     super().__init__()
#     self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
#     self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)


#   def forward(self, x, edge_index):
#     h = F.dropout(x, p=0.6, training=self.training)
#     h = self.gat1(x, edge_index)
#     h = F.relu(h)
#     h = F.dropout(h, p=0.6, training=self.training)
#     h = self.gat2(h, edge_index)
#     h= F.sigmoid(h)
#     return h

# 모델 커스터마이징

In [34]:
import torch
torch.manual_seed(-1)
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
import torch.nn as nn
from sklearn.metrics import f1_score, accuracy_score
from torch_geometric.nn import GIN, GAT

class customSAGE(torch.nn.Module):
    
    def __init__(self, in_channels, hidden_channels, num_layers,out_channels):
        super(customSAGE,self).__init__()
        self.sage = GAT(in_channels=in_channels,hidden_channels=hidden_channels,num_layers=num_layers,out_channels=out_channels)
        # self.fc1 = nn.Linear(out_channels,out_channels)
    def forward(self,data):
        sage_output = self.sage(x = data.x, edge_index= data.edge_index, edge_attr=data.edge_attr)
        # x = torch.relu(sage_output)
        # x = self.fc1(x)
        return sage_output
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = customSAGE(in_channels=cfg.in_dim,hidden_channels=cfg.h_dim,num_layers=cfg.num_layers,out_channels=cfg.out_dim).to(device)

# 모델 output 확인

In [35]:
# model(lst[0].to(device))

# Train

In [36]:
# def accuracy(pred_y, y):
#     """정확도 계산"""
#     return ((pred_y == y).sum() / len(y)).item()


# def train():
#     model.train()

#     total_loss = 0
#     for data in train_loader:
        
#         data = data.to(device)
#         optimizer.zero_grad()
        
#         out = model(data.x, data.edge_index)
#         loss = criterion(out, data.y)
#         # total_loss += loss.item() * data.num_graphs
#         loss.backward()
#         optimizer.step()
        
#     return total_loss / len(train_loader.dataset)



# @torch.no_grad()
# def test(loader):
#     model.eval()
    
#     ys, preds = [], [] 
    
#     for data in loader:
        
#         out = model(data.x.to(device), data.edge_index.to(device))
#         out = torch.argmax(out, dim=1)
#         preds.append((out > 0).float().cpu())
        
#         y, pred = torch.cat(ys, dim=0).numpy().squeeze(), torch.cat(preds, dim=0).numpy()
#     print(y.sum(), pred.sum())
    
#     return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0

In [37]:
# from tqdm import tqdm
# for epoch in tqdm(range(10)):
#     loss = fit()
#     val_f1 = test(test_loader)
#     if epoch % 1 == 0:
#         print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Test F1-score: {val_f1:.4f}')
#         # print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Val F1-score: {val_f1:.4f}')

# # print(f'Test F1-score: {test(test_loader):.4f}')

In [38]:
def train(loader):
    
    model.train()
    total_loss = 0
    
    for i,data in enumerate(loader,0):
        
        data = data.to(device)
        optimizer.zero_grad()
        
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss/ len(loader)
    

def test(loader):
    

    model.eval()
    predict = list()
    actual = list()
    with torch.no_grad():
        for _, data in enumerate(loader,0):
            data = data.to(device)
            out = model(data)
            m = nn.Sigmoid()
            preds = (m(out)>0.5).float().cpu().numpy()
            y = data.y.cpu().numpy()
            
            predict.append(preds)
            actual.append(y)
            
    predictions, actuals = vstack(predict), vstack(actual)
    acc = accuracy_score(actuals, predictions),f1_score(actuals, predictions,average='micro')

            # total += sum(out==data.y)/len(data.y)
            # print(sum(out==data.y)/len(data.y))
            # cnt +=1
    return acc 

In [39]:
from tqdm import tqdm
from numpy import vstack


# model = GraphSAGE(dim_in=lst[0].num_features,dim_h=128,dim_out=1).to(device)
# model = GAT(dim_in=lst[0].num_features,dim_h=64,dim_out=1).to(device)
criterion = torch.nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)

for epoch in tqdm(range(300)):
    loss = train(train_loader)
    acc,f1 = test(test_loader)
    if epoch %20 == 0:
        print(f'Train Loss: {loss:.3f} | Test acc: {acc:.3f} | Test f1: {f1:.3f} ')
        # print(f'Train Loss: {loss:.3f} | Test acc: {acc.item():.3f} ')
        # print(f'Train Loss: {loss:.3f}')
    
    

  0%|          | 1/300 [00:00<02:21,  2.11it/s]

Train Loss: 18647.857 | Test acc: 0.534 | Test f1: 0.534 


  7%|▋         | 21/300 [00:09<02:12,  2.11it/s]

Train Loss: 55.677 | Test acc: 0.484 | Test f1: 0.484 


 14%|█▎        | 41/300 [00:19<01:55,  2.25it/s]

...

 87%|████████▋ | 261/300 [02:06<00:18,  2.05it/s]

Train Loss: 0.917 | Test acc: 0.640 | Test f1: 0.640 


 94%|█████████▎| 281/300 [02:16<00:09,  2.06it/s]

Train Loss: 0.863 | Test acc: 0.574 | Test f1: 0.574 


100%|██████████| 300/300 [02:25<00:00,  2.06it/s]


In [46]:
lst[0].is_undirected()

False

# example

In [ ]:
# import torch
# from sklearn.metrics import f1_score
# from torch_geometric.nn import GraphSAGE
 
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
# model = GraphSAGE(
#     in_channels=train_dataset.num_features,
#     hidden_channels=512,
#     num_layers=2,
#     out_channels=train_dataset.num_classes,
# ).to(device)

In [ ]:
# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
 
# def fit():
#     model.train()
 
#     total_loss = 0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
        
#         out = model(data.x, data.edge_index)
#         loss = criterion(out, data.y)
        
#         total_loss += loss.item() * data.num_graphs
        
#         loss.backward()
#         optimizer.step()
        
#     return total_loss / len(train_loader.dataset)
 
# @torch.no_grad()
# def test(loader):
#     model.eval()
 
#     data = next(iter(loader))
#     out = model(data.x.to(device), data.edge_index.to(device))
#     preds = (out > 0).float().cpu()
#     print(preds)
 
#     y, pred = data.y.numpy(), preds.numpy()
#     return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0
 
# for epoch in range(301):
#     loss = fit()
#     val_f1 = test(val_loader)
#     if epoch % 50 == 0:
#         print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Val F1-score: {val_f1:.4f}')
 
# print(f'Test F1-score: {test(test_loader):.4f}')

In [ ]:
# import torch

# def test(test_loader):
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for data in test_loader:
#             data = data.to(device)
#             out = model(data)
#             preds = (out > 0).float().cpu()  # .cpu()를 추가하여 GPU에서 CPU로 이동
#             acc = accuracy_score(data.y.cpu(), preds)  # .cpu()를 추가하여 GPU에서 CPU로 이동
#             print(acc)
#             total += len(data.y)
#             correct += (preds == data.y.cpu()).sum().item()
#     return correct / total

# # 학습 및 테스트 코드
# for epoch in tqdm(range(100)):
#     loss = train(train_loader)
#     acc = test(test_loader)
#     if epoch % 1 == 0:
#         print(f'Train Loss: {loss:.3f} | Test acc: {acc:.3f}')
